# GGUF 和 Transformer 的交互

GGUF 文件格式用于存储与 [GGML](https://github.com/ggerganov/ggml) 及其依赖库（如非常流行的 [llama.cpp](https://github.com/ggerganov/llama.cpp) 或 [whisper.cpp](https://github.com/ggerganov/whisper.cpp)）一起进行推理的模型。

这是一种由 [Hugging Face Hub 支持的文件格式](https://huggingface.co/docs/hub/en/gguf)，具有快速检查文件中张量和元数据的功能。

这种文件格式设计为“单文件格式”，通常一个文件中包含配置属性、分词器词汇表和其他属性，以及要加载到模型中的所有张量。这些文件根据文件的量化类型有不同的格式。我们在这里简要介绍一些常见的格式：[量化类型](https://huggingface.co/docs/hub/en/gguf#quantization-types)。

## Transformer 中的支持

我们在 `transformers` 中添加了加载 `gguf` 文件的能力，以提供对 gguf 模型进一步训练或微调的能力，并在完成后再将这些模型转换回 `gguf` 格式，以便在 `ggml` 生态系统中使用。加载模型时，我们会先将其去量化为 fp32，然后再加载权重以供 PyTorch 使用。

> **注意**：当前的支持仍处于探索性阶段，我们欢迎贡献以巩固不同量化类型和模型架构的支持。

目前支持的模型架构和量化类型如下：

### 支持的量化类型

初始支持的量化类型是根据在 Hub 上共享的流行量化文件决定的。

- F32
- F16
- BF16
- Q4_0
- Q4_1
- Q5_0
- Q5_1
- Q8_0
- Q2_K
- Q3_K
- Q4_K
- Q5_K
- Q6_K
- IQ1_S
- IQ1_M
- IQ2_XXS
- IQ2_XS
- IQ2_S
- IQ3_XXS
- IQ3_S
- IQ4_XS
- IQ4_NL

> **注意**：为了支持 gguf 去量化，需要安装 `gguf>=0.10.0`。

### 支持的模型架构

目前支持的模型架构是在 Hub 上非常流行的架构，包括：

- LLaMa
- Mistral
- Qwen2
- Qwen2Moe
- Phi3
- Bloom
- Falcon
- StableLM
- GPT2
- Starcoder2

## 示例用法

要在 `transformers` 中加载 `gguf` 文件，您需要在 `from_pretrained` 方法中指定 `gguf_file` 参数，用于加载分词器和模型。以下是如何加载分词器和模型的示例，它们可以从同一个文件中加载：


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
filename = "tinyllama-1.1b-chat-v1.0.Q6_K.gguf"

tokenizer = AutoTokenizer.from_pretrained(model_id, gguf_file=filename)
model = AutoModelForCausalLM.from_pretrained(model_id, gguf_file=filename)


现在，您可以在 PyTorch 生态系统中访问完整的去量化模型，并结合其他多种工具使用。

为了将模型重新转换为 `gguf` 文件，我们推荐使用 llama.cpp 中的 [`convert-hf-to-gguf.py` 脚本](https://github.com/ggerganov/llama.cpp/blob/master/convert_hf_to_gguf.py)。

以下是完成上述脚本以保存模型并将其导出为 `gguf` 文件的方法：


In [ ]:
tokenizer.save_pretrained('directory')
model.save_pretrained('directory')

In [ ]:
!python ${path_to_llama_cpp}/convert-hf-to-gguf.py ${directory}